In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/embedding-work/combined_embedding.csv
/kaggle/input/df-main/df_main.csv


In [2]:
import torch

In [3]:
from transformers import pipeline

pipe = pipeline("text-classification", model="phoner45/wangchan-sentiment-thai-text-model")

2025-05-06 09:23:45.567998: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746523425.596952     165 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746523425.605580     165 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Device set to use cpu


In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

tokenizer = AutoTokenizer.from_pretrained("phoner45/wangchan-sentiment-thai-text-model")
model = AutoModel.from_pretrained("phoner45/wangchan-sentiment-thai-text-model")

Some weights of CamembertModel were not initialized from the model checkpoint at phoner45/wangchan-sentiment-thai-text-model and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
df = pd.read_csv("/kaggle/input/df-main/df_main.csv")
df.head(2)

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,img_id,sentences,text_length,word_list,Negative,Neutral,Positive,has_cta,ctr,ctr_group
0,0.708480,0.759826,0.449038,0.476864,0.791149,0.541827,0.389004,0.282455,0.802612,0.318787,...,c7e254ff6e829bbd379c791c06dba26a,วันนี - 14 มีนาคม 2568 รีบด่วน 400 web hosting,46,"['วัน', 'นี', ' ', '-', ' ', '14', ' ', 'มีนาค...",0,16,0,False,0.000000,Low
1,0.733801,0.642118,0.425671,0.490838,0.708187,0.605140,0.204538,0.374215,0.633562,0.262923,...,310f05853aa67594eadcecfaeab260c9,แฟลชเซล มาก่อน ได้ก่อน 899 12 00 น.,35,"['แฟลช', 'เซ', 'ล', ' ', 'มา', 'ก่อน', ' ', 'ไ...",0,1,0,False,0.016482,Low


In [6]:
def get_bert_embedding(text):
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        output = model(**tokens)
    embedding = output.last_hidden_state.mean(dim=1).squeeze().numpy()
    return embedding

In [7]:
df['sentence_embeddings'] = df['sentences'].apply(get_bert_embedding)

In [8]:
df['word_embeddings'] = df['word_list'].apply(get_bert_embedding)

In [9]:
df.columns

Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       ...
       'text_length', 'word_list', 'Negative', 'Neutral', 'Positive',
       'has_cta', 'ctr', 'ctr_group', 'sentence_embeddings',
       'word_embeddings'],
      dtype='object', length=141)

In [10]:
# df.to_csv('combined_embedding.csv', index=False)

In [11]:
import ast
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
sent_embed = np.stack(df['sentence_embeddings'].values)
word_embed = np.stack(df['word_embeddings'].values)

In [13]:
cos_sim = np.array([
    cosine_similarity(sent_embed[i].reshape(1, -1), word_embed[i].reshape(1, -1))[0][0]
    for i in range(len(df))
])

df['embedding_cos_sim'] = cos_sim

In [14]:
print(type(df['sentence_embeddings'][0]), len(df['sentence_embeddings'][0]))

<class 'numpy.ndarray'> 768


In [15]:
df['embedding_cos_sim'] = cos_sim

In [16]:
df['embedding_cos_sim']

0       0.536670
1       0.502119
2       0.574336
3       0.507568
4       0.620794
          ...   
1110    0.799550
1111    0.660865
1112    0.599210
1113    0.462548
1114    0.441794
Name: embedding_cos_sim, Length: 1115, dtype: float32

In [17]:
df

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,...,word_list,Negative,Neutral,Positive,has_cta,ctr,ctr_group,sentence_embeddings,word_embeddings,embedding_cos_sim
0,0.708480,0.759826,0.449038,0.476864,0.791149,0.541827,0.389004,0.282455,0.802612,0.318787,...,"['วัน', 'นี', ' ', '-', ' ', '14', ' ', 'มีนาค...",0,16,0,False,0.000000,Low,"[1.9249656, -1.1482828, -0.25647768, -0.198943...","[1.4809455, -0.124824405, -0.8571908, 1.193851...",0.536670
1,0.733801,0.642118,0.425671,0.490838,0.708187,0.605140,0.204538,0.374215,0.633562,0.262923,...,"['แฟลช', 'เซ', 'ล', ' ', 'มา', 'ก่อน', ' ', 'ไ...",0,1,0,False,0.016482,Low,"[0.5988743, -0.8474121, -0.8484417, 0.3760249,...","[1.0963709, 0.13514143, -0.7340203, 1.1285696,...",0.502119
2,0.587095,0.655419,0.328038,0.582514,0.626008,0.675075,0.503859,0.556132,0.578276,0.380239,...,"['ifex', ' ', 'prc', ' ', 'ติว', 'เข้ม', ' ', ...",0,6,0,False,0.000000,Low,"[-0.15545496, -0.94289625, -0.40212044, 1.1409...","[0.8593514, -0.6384615, -1.1777972, 1.9251543,...",0.574336
3,0.353129,0.343184,0.522671,0.418777,0.223161,0.525447,0.458935,0.600709,0.323766,0.623137,...,"['สร้าง', 'รายได้', 'เสริม', 'จาก', 'ที่', 'บ้...",0,37,0,True,0.000000,Low,"[0.54072624, -0.33284494, 0.52688557, -0.26157...","[0.9857843, -0.5182989, -0.6827191, 1.3789423,...",0.507568
4,0.570452,0.067015,0.308653,0.352218,0.347666,0.537102,0.193613,0.497158,0.483621,0.717515,...,"['uniol', '0', ' ', 'management', ' ', 'candid...",0,17,0,True,0.000000,Low,"[0.9832803, -0.8853229, 0.201369, 0.26472697, ...","[0.8781025, -0.59901005, -0.8466322, 1.1779596...",0.620794
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1110,0.501477,0.709318,0.594054,0.468367,0.402899,0.624929,0.440527,0.272052,0.834530,0.241055,...,"['empty', '_record']",0,0,0,False,0.072270,High,"[-0.54987603, 0.057413094, 0.005992517, -0.036...","[0.55158556, -0.43008462, -0.42940703, 0.38347...",0.799550
1111,0.339184,0.468420,0.391167,0.396819,0.314405,0.466089,0.555852,0.497243,0.397880,0.577542,...,"['spring', ' ', 'sale']",0,70,0,False,0.053965,High,"[-0.14109027, -0.5164167, -0.94308466, 0.48717...","[0.90333956, -0.8059393, -1.0364352, 0.6873728...",0.660865
1112,0.573479,0.587674,0.394277,0.786340,0.636821,0.249928,0.536042,0.363466,0.418655,0.417026,...,['cstu'],0,0,0,False,0.063264,High,"[-0.6419372, 0.081582785, 0.5833726, 0.7918693...","[1.4488207, -0.74437934, -0.1535783, 0.8936018...",0.599210
1113,0.662974,0.529609,0.232303,0.705959,0.504155,0.368674,0.386872,0.190539,0.321462,0.451873,...,"['คณะ', 'สกิ', 'ติ', 'ประยุกต์', 'นิ', 'ดา', '...",0,35,0,False,0.000000,Low,"[0.011391203, -0.22755602, -0.2921897, 0.28127...","[1.1329646, -0.21876517, -0.5761984, 1.2961178...",0.462548


In [18]:
df.columns

Index(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       ...
       'word_list', 'Negative', 'Neutral', 'Positive', 'has_cta', 'ctr',
       'ctr_group', 'sentence_embeddings', 'word_embeddings',
       'embedding_cos_sim'],
      dtype='object', length=142)

In [22]:
latent_vectors = df[[f'feature_{i}' for i in range(128)]].values
word_embeddings = np.vstack(df['word_embeddings'].values)

In [326]:
from sklearn.decomposition import PCA

pca = PCA(n_components=128)
word_embeddings_reduced = pca.fit_transform(word_embeddings)

dot_products = np.sum(latent_vectors * word_embeddings_reduced, axis=1)
df['latent_dot_word_emb'] = dot_products

In [327]:
# umap_reducer = umap.UMAP(n_neighbors=50, min_dist=0.001, n_components=10, metric='cosine', random_state=42)
# latent_umap = umap_reducer.fit_transform(latent_vectors)

In [366]:
elementwise_multiplication = latent_vectors * word_embeddings_reduced
sum_elementwise_multiplication = np.sum(elementwise_multiplication, axis=1)
df['latent_elementwise_mul_word_emd'] = sum_elementwise_multiplication

In [367]:
absolute_difference = np.abs(latent_vectors - word_embeddings_reduced)
sum_absolute_difference = np.sum(absolute_difference, axis=1)
df['latent_abs_diff_word_emb'] = sum_absolute_difference

In [24]:
latent_mean = latent_vectors.mean(axis=1)
latent_std = latent_vectors.std(axis=1)
latent_max = latent_vectors.max(axis=1)
latent_min = latent_vectors.min(axis=1)

In [25]:
latent_padded = np.pad(latent_vectors, ((0, 0), (0, 640)), mode='constant')  # 128 → 768

In [26]:
n_samples = 1115

In [27]:
interaction_cos_sim = np.array([
    cosine_similarity(latent_padded[i].reshape(1, -1), word_embeddings[i].reshape(1, -1))[0][0]
    for i in range(n_samples)
])

In [369]:
latent_v = df.filter(like='feature_').values

df_features = pd.DataFrame(latent_v, columns=[f'latent_vector_{i}' for i in range(128)])
df_features['latent_mean'] = latent_mean
df_features['latent_std'] = latent_std
df_features['latent_max'] = latent_max
df_features['latent_min'] = latent_min
df_features['latent_word_cos_sim'] = interaction_cos_sim

In [ ]:
df_meta    = pd.DataFrame(meta_features, columns=['cluster_id','Positive','Negative','Neutral'], index=df.index)
df_cosim   = pd.DataFrame(embed_cosim.ravel(), columns=['embed_cosim'], index=df.index)
df_word    = pd.DataFrame(word_array, columns=[f'word_emb_{i}' for i in range(word_array.shape[1])], index=df.index)

In [370]:
df_features = pd.concat([df_features, df_word, df_meta, df_cosim, df['latent_dot_word_emb'], df['latent_elementwise_mul_word_emd'], df['latent_abs_diff_word_emb']], axis=1)

In [371]:
from sklearn.preprocessing import StandardScaler

std = StandardScaler()
x_scaled = std.fit_transform(df_features)

In [46]:
# df['ctr_group'] = df['ctr_group'].map({'Low': 0, 'High': 1})

In [372]:
from sklearn.model_selection import train_test_split

X = x_scaled
y = df['ctr_group']

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

In [373]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((892, 909), (892,), (111, 909), (111,), (112, 909), (112,))

In [374]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

In [375]:
print(type(X_train))
print(np.array(X_train).shape)

<class 'numpy.ndarray'>
(892, 909)


In [430]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

In [409]:
params = {
    'objective': 'binary:logistic',  # Classification binary
    'eval_metric': 'logloss',        # ใช้ logloss เป็น metric
    'max_depth': 10,                 # ความลึกสูงสุดของต้นไม้
    'learning_rate': 0.05,           # อัตราการเรียนรู้
    # 'n_estimators': 2000,            # จำนวนต้นไม้
    'subsample': 0.8,                # ใช้ 80% ของข้อมูลในการฝึก
    'colsample_bytree': 0.8,         # ใช้ 80% ของฟีเจอร์ในการฝึก
    'gamma': 0.1,                    # การควบคุมการ split ของต้นไม้
    'scale_pos_weight': 1.0          # ถ้ามี imbalance ใน class
}

In [440]:
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 10,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'gamma': 0.1,
    'scale_pos_weight': 1.0,
    'min_child_weight': 5,
    'lambda': 1.0,
    'alpha': 0.5,
    'tree_method': 'hist',
    'max_bin': 128,
    'sampling_method': 'uniform'
}

In [398]:
# params = {
#     'objective': 'binary:logistic',   # ยังเหมาะสม
#     'eval_metric': 'logloss',         # เหมาะสำหรับ binary classification
#     'max_depth': 3,                   # ลดความลึกเพื่อลด overfitting
#     'learning_rate': 0.1,             # เพิ่ม learning rate ให้เรียนรู้ไวขึ้น
#     # 'n_estimators': 200,              # ลดจำนวนต้นไม้ลง
#     'subsample': 0.8,                 # เหมาะสม
#     'colsample_bytree': 0.8,          # เหมาะสม
#     'gamma': 0,                       # ปรับเป็น 0 ก่อน หาก overfitting ค่อยเพิ่ม
#     'scale_pos_weight': 1.0           # ยังเหมาะ ถ้า class balance
# }

In [441]:
evals = [(dtrain, 'train'), (dval, 'eval')]

In [442]:
num_round = 1000  
early_stopping_rounds = 50 

In [443]:
evals_result = {}

In [444]:
bst = xgb.train(
    params, 
    dtrain, 
    num_round, 
    evals=evals, 
    early_stopping_rounds=early_stopping_rounds,
    evals_result=evals_result,  
    verbose_eval=True
)

[0]	train-logloss:0.66167	eval-logloss:0.66514
[1]	train-logloss:0.63269	eval-logloss:0.63730
[2]	train-logloss:0.60524	eval-logloss:0.61576
[3]	train-logloss:0.58014	eval-logloss:0.59104
[4]	train-logloss:0.55778	eval-logloss:0.57133
[5]	train-logloss:0.53626	eval-logloss:0.55198
[6]	train-logloss:0.51567	eval-logloss:0.53553
[7]	train-logloss:0.49591	eval-logloss:0.51771
[8]	train-logloss:0.47777	eval-logloss:0.50262
[9]	train-logloss:0.46065	eval-logloss:0.48812
[10]	train-logloss:0.44479	eval-logloss:0.47281
[11]	train-logloss:0.42989	eval-logloss:0.46070
[12]	train-logloss:0.41586	eval-logloss:0.44870
[13]	train-logloss:0.40239	eval-logloss:0.43523
[14]	train-logloss:0.38940	eval-logloss:0.42508
[15]	train-logloss:0.37682	eval-logloss:0.41548
[16]	train-logloss:0.36464	eval-logloss:0.40707
[17]	train-logloss:0.35371	eval-logloss:0.39813
[18]	train-logloss:0.34319	eval-logloss:0.39061
[19]	train-logloss:0.33324	eval-logloss:0.38440
[20]	train-logloss:0.32385	eval-logloss:0.37678
[2

In [447]:
y_pred = bst.predict(dval)
y_pred_binary = (y_pred > 0.5).astype(int)  # binary class

In [448]:
accuracy = accuracy_score(y_val, y_pred_binary)
print(f"Validation Accuracy: {accuracy:.4f}")

Validation Accuracy: 0.9009


In [449]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss

In [452]:
train_losses = []
val_losses = []

In [451]:
rf = RandomForestClassifier(
    n_estimators=150,          # ลดลงเพื่อลดเวลาเทรน + ใช้ early stopping ช่วย
    max_depth=6,               # เพิ่มขึ้นเล็กน้อยให้เรียนรู้ complex pattern ได้มากขึ้น
    max_features=0.3,          # ใช้แค่ 30% ของฟีเจอร์เพื่อเพิ่ม diversity
    min_samples_split=5,       # ลดลงเพื่อให้ tree กล้า split มากขึ้น
    min_samples_leaf=3,        # ลดขนาดใบไม้ให้ต้นไม้ลึกขึ้นนิด ๆ แต่ยังควบคุม overfit
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

In [454]:
early_stop_patience = 5
best_val_loss = float('inf')
no_improve_count = 0
early_stop_after = 30  

In [455]:
for n in range(10, 210, 10):  
    rf.set_params(n_estimators=n)
    rf.fit(X_train, y_train)

    y_train_pred_proba = rf.predict_proba(X_train)
    y_val_pred_proba = rf.predict_proba(X_val)

    train_logloss = log_loss(y_train, y_train_pred_proba)
    val_logloss = log_loss(y_val, y_val_pred_proba)

    train_losses.append(train_logloss)
    val_losses.append(val_logloss)

    print(f"Trees: {n} - Train Loss: {train_logloss:.4f} | Val Loss: {val_logloss:.4f}")

    if n > early_stop_after:
        if val_logloss < best_val_loss:
            best_val_loss = val_logloss
            no_improve_count = 0
        else:
            no_improve_count += 1

        if no_improve_count >= early_stop_patience:
            print(f"⛔️ Early stopping at {n} trees (no improvement in {early_stop_patience} rounds after 30 trees).")
            break

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:583: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


Trees: 10 - Train Loss: 0.1143 | Val Loss: 0.3056
Trees: 20 - Train Loss: 0.1112 | Val Loss: 0.3070
Trees: 30 - Train Loss: 0.1109 | Val Loss: 0.2962
Trees: 40 - Train Loss: 0.1126 | Val Loss: 0.2945
Trees: 50 - Train Loss: 0.1111 | Val Loss: 0.3023
Trees: 60 - Train Loss: 0.1118 | Val Loss: 0.2979
Trees: 70 - Train Loss: 0.1103 | Val Loss: 0.2970
Trees: 80 - Train Loss: 0.1104 | Val Loss: 0.2958
Trees: 90 - Train Loss: 0.1108 | Val Loss: 0.2916
Trees: 100 - Train Loss: 0.1101 | Val Loss: 0.2913
Trees: 110 - Train Loss: 0.1102 | Val Loss: 0.2900
Trees: 120 - Train Loss: 0.1093 | Val Loss: 0.2901
Trees: 130 - Train Loss: 0.1096 | Val Loss: 0.2891
Trees: 140 - Train Loss: 0.1099 | Val Loss: 0.2911
Trees: 150 - Train Loss: 0.1093 | Val Loss: 0.2888
Trees: 160 - Train Loss: 0.1092 | Val Loss: 0.2885
Trees: 170 - Train Loss: 0.1087 | Val Loss: 0.2884
Trees: 180 - Train Loss: 0.1086 | Val Loss: 0.2891
Trees: 190 - Train Loss: 0.1090 | Val Loss: 0.2872
Trees: 200 - Train Loss: 0.1090 | Val Lo

In [458]:
kg = rf.predict(X_test)
accuracy_score(y_test, kg)

0.8125